# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f930ccda9a0>
├── 'a' --> tensor([[ 0.0962,  1.2162,  1.6858],
│                   [ 1.5490, -0.6782, -1.1283]])
└── 'x' --> <FastTreeValue 0x7f930ccdac10>
    └── 'c' --> tensor([[-0.7926,  0.5028, -1.5390,  0.6423],
                        [-0.7281,  0.4362,  1.3483,  0.7270],
                        [-1.0876, -1.2192,  0.4863, -1.8572]])

In [4]:
t.a

tensor([[ 0.0962,  1.2162,  1.6858],
        [ 1.5490, -0.6782, -1.1283]])

In [5]:
%timeit t.a

79.6 ns ± 0.235 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f930ccda9a0>
├── 'a' --> tensor([[ 0.8836, -0.2391, -0.0963],
│                   [ 2.2318,  0.1842,  0.2478]])
└── 'x' --> <FastTreeValue 0x7f930ccdac10>
    └── 'c' --> tensor([[-0.7926,  0.5028, -1.5390,  0.6423],
                        [-0.7281,  0.4362,  1.3483,  0.7270],
                        [-1.0876, -1.2192,  0.4863, -1.8572]])

In [7]:
%timeit t.a = new_value

82.9 ns ± 0.0513 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0962,  1.2162,  1.6858],
               [ 1.5490, -0.6782, -1.1283]]),
    x: Batch(
           c: tensor([[-0.7926,  0.5028, -1.5390,  0.6423],
                      [-0.7281,  0.4362,  1.3483,  0.7270],
                      [-1.0876, -1.2192,  0.4863, -1.8572]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0962,  1.2162,  1.6858],
        [ 1.5490, -0.6782, -1.1283]])

In [11]:
%timeit b.a

71.1 ns ± 0.0698 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.5134,  0.7193, -0.5079],
               [-0.0279, -0.3575,  0.9507]]),
    x: Batch(
           c: tensor([[-0.7926,  0.5028, -1.5390,  0.6423],
                      [-0.7281,  0.4362,  1.3483,  0.7270],
                      [-1.0876, -1.2192,  0.4863, -1.8572]]),
       ),
)

In [13]:
%timeit b.a = new_value

598 ns ± 1.09 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1 µs ± 1.74 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12 µs ± 86.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

165 µs ± 844 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f925f9a7f10>
├── 'a' --> tensor([[[ 0.0962,  1.2162,  1.6858],
│                    [ 1.5490, -0.6782, -1.1283]],
│           
│                   [[ 0.0962,  1.2162,  1.6858],
│                    [ 1.5490, -0.6782, -1.1283]],
│           
│                   [[ 0.0962,  1.2162,  1.6858],
│                    [ 1.5490, -0.6782, -1.1283]],
│           
│                   [[ 0.0962,  1.2162,  1.6858],
│                    [ 1.5490, -0.6782, -1.1283]],
│           
│                   [[ 0.0962,  1.2162,  1.6858],
│                    [ 1.5490, -0.6782, -1.1283]],
│           
│                   [[ 0.0962,  1.2162,  1.6858],
│                    [ 1.5490, -0.6782, -1.1283]],
│           
│                   [[ 0.0962,  1.2162,  1.6858],
│                    [ 1.5490, -0.6782, -1.1283]],
│           
│                   [[ 0.0962,  1.2162,  1.6858],
│                    [ 1.5490, -0.6782, -1.1283]]])
└── 'x' --> <FastTreeValue 0x7f930cd3bd90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.5 µs ± 59.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f930ccd57f0>
├── 'a' --> tensor([[ 0.0962,  1.2162,  1.6858],
│                   [ 1.5490, -0.6782, -1.1283],
│                   [ 0.0962,  1.2162,  1.6858],
│                   [ 1.5490, -0.6782, -1.1283],
│                   [ 0.0962,  1.2162,  1.6858],
│                   [ 1.5490, -0.6782, -1.1283],
│                   [ 0.0962,  1.2162,  1.6858],
│                   [ 1.5490, -0.6782, -1.1283],
│                   [ 0.0962,  1.2162,  1.6858],
│                   [ 1.5490, -0.6782, -1.1283],
│                   [ 0.0962,  1.2162,  1.6858],
│                   [ 1.5490, -0.6782, -1.1283],
│                   [ 0.0962,  1.2162,  1.6858],
│                   [ 1.5490, -0.6782, -1.1283],
│                   [ 0.0962,  1.2162,  1.6858],
│                   [ 1.5490, -0.6782, -1.1283]])
└── 'x' --> <FastTreeValue 0x7f930ccd5190>
    └── 'c' --> tensor([[-0.7926,  0.5028, -1.5390,  0.6423],
                        [-0.7281,  0.4362,  1.3483,  0.7270],
                 

In [23]:
%timeit t_cat(trees)

35.8 µs ± 69.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.6 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0962,  1.2162,  1.6858],
                [ 1.5490, -0.6782, -1.1283]],
       
               [[ 0.0962,  1.2162,  1.6858],
                [ 1.5490, -0.6782, -1.1283]],
       
               [[ 0.0962,  1.2162,  1.6858],
                [ 1.5490, -0.6782, -1.1283]],
       
               [[ 0.0962,  1.2162,  1.6858],
                [ 1.5490, -0.6782, -1.1283]],
       
               [[ 0.0962,  1.2162,  1.6858],
                [ 1.5490, -0.6782, -1.1283]],
       
               [[ 0.0962,  1.2162,  1.6858],
                [ 1.5490, -0.6782, -1.1283]],
       
               [[ 0.0962,  1.2162,  1.6858],
                [ 1.5490, -0.6782, -1.1283]],
       
               [[ 0.0962,  1.2162,  1.6858],
                [ 1.5490, -0.6782, -1.1283]]]),
    x: Batch(
           c: tensor([[[-0.7926,  0.5028, -1.5390,  0.6423],
                       [-0.7281,  0.4362,  1.3483,  0.7270],
                       [-1.0876, -1.2192,  0.4863, -1.8572]],
         

In [26]:
%timeit Batch.stack(batches)

92.5 µs ± 66.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0962,  1.2162,  1.6858],
               [ 1.5490, -0.6782, -1.1283],
               [ 0.0962,  1.2162,  1.6858],
               [ 1.5490, -0.6782, -1.1283],
               [ 0.0962,  1.2162,  1.6858],
               [ 1.5490, -0.6782, -1.1283],
               [ 0.0962,  1.2162,  1.6858],
               [ 1.5490, -0.6782, -1.1283],
               [ 0.0962,  1.2162,  1.6858],
               [ 1.5490, -0.6782, -1.1283],
               [ 0.0962,  1.2162,  1.6858],
               [ 1.5490, -0.6782, -1.1283],
               [ 0.0962,  1.2162,  1.6858],
               [ 1.5490, -0.6782, -1.1283],
               [ 0.0962,  1.2162,  1.6858],
               [ 1.5490, -0.6782, -1.1283]]),
    x: Batch(
           c: tensor([[-0.7926,  0.5028, -1.5390,  0.6423],
                      [-0.7281,  0.4362,  1.3483,  0.7270],
                      [-1.0876, -1.2192,  0.4863, -1.8572],
                      [-0.7926,  0.5028, -1.5390,  0.6423],
                      [-0.7281,  

In [28]:
%timeit Batch.cat(batches)

168 µs ± 225 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

444 µs ± 26.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
